In [ ]:
#necessary installation
!pip install open3d
!pip install open3d-cpu

In [ ]:
import open3d as o3d
import numpy as np
print(o3d.__version__)


In [3]:
#path of the .ply file
path="/content/shoe_pc.ply"

In [ ]:
# reading the point cloud
pcd_point_cloud = o3d.data.PCDPointCloud()
pcd = o3d.io.read_point_cloud(path)


In [7]:
#detecting the plane
plane_model, inliers = pcd.segment_plane(distance_threshold=0.04,
                                         ransac_n=3,
                                         num_iterations=1000)

In [8]:
#fetching plane parameters
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

Plane equation: 0.31x + -0.00y + 0.95z + -1.34 = 0


In [18]:
#inclier cloud contains the points representing the floor(red colour). file name floor.ply
#outlier cloud contains the points representing the shoe(blue colour). file name object.ply
inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
o3d.io.write_point_cloud("floor.ply", inlier_cloud)
outlier_cloud = pcd.select_by_index(inliers, invert=True)
outlier_cloud, ind = outlier_cloud.remove_radius_outlier(nb_points=20, radius=0.05)
outlier_cloud.paint_uniform_color([0, 0, 1.0])
o3d.io.write_point_cloud("object.ply", outlier_cloud)

[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [19]:
#calculate the center of the plane
mid, covar=inlier_cloud.compute_mean_and_covariance()
center=[0,0,0]

In [ ]:
#compute the translate vector
translate_vector= center-mid
print(translate_vector)

In [ ]:
#apply translation to take it to the center
plane_translate=inlier_cloud.translate(translate_vector, relative=True)
o3d.io.write_point_cloud("plane_translate.ply", plane_translate)
object_translate=outlier_cloud.translate(translate_vector, relative=True)
o3d.io.write_point_cloud("object_translate.ply", object_translate)


In [ ]:
#computing the normals of the planes
plane_normal=np.array(plane_model[0:3])
target_normal=np.array([0.0,1.0,0.0])
print(plane_normal)
print(target_normal)

In [23]:
#rotation matrix computation
def rotation_matrix(plane_A_normal, plane_B_normal):
    plane_A_normal /= np.linalg.norm(plane_A_normal)
    plane_B_normal /= np.linalg.norm(plane_B_normal)

    # Compute the axis of rotation and angle of rotation
    axis = np.cross(plane_A_normal, plane_B_normal)
    angle = np.arccos(np.dot(plane_A_normal, plane_B_normal))

    # Compute the rotation matrix using Rodrigues' rotation formula
    cos_theta = np.cos(angle)
    sin_theta = np.sin(angle)
    cross_matrix = np.array([[0, -axis[2], axis[1]],
                             [axis[2], 0, -axis[0]],
                             [-axis[1], axis[0], 0]])
    rotation_matrix = np.eye(3) + sin_theta * cross_matrix + (1 - cos_theta) * np.dot(cross_matrix, cross_matrix)

    return rotation_matrix

In [26]:
#rotation angle
R = rotation_matrix(plane_normal,target_normal)

In [25]:
R

array([[ 0.90681009, -0.30500126, -0.29098007],
       [ 0.30500126, -0.00175826,  0.95234869],
       [-0.29098007, -0.95234869,  0.09143165]])

In [27]:
# rotating the floor and the shoe
plane_rotate=plane_translate.rotate(R,center)
object_rotate=object_translate.rotate(R,center)

In [29]:
#saving the rotated floor and object in floor_rotate.ply and object_rotate.ply
o3d.io.write_point_cloud("object_rotate.ply", object_rotate)
o3d.io.write_point_cloud("floor_rotate.ply", plane_rotate)


True

In [31]:
# mesh creation floor_mesh.ply and object_mesh.ply
alpha = 0.04
plane_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(plane_rotate, alpha)
object_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(object_rotate, alpha)

o3d.io.write_triangle_mesh("floor_mesh.ply", plane_mesh)
o3d.io.write_triangle_mesh("object_mesh.ply", object_mesh)

True